# Results Visualization
Comparing Full Fine-tuning vs LoRA on Financial Sentiment Classification

In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_style("whitegrid")
plt.rcParams['font.size'] = 10

In [ ]:
# Load results
with open("full_finetuning_results/results_summary.json") as f:
    full_results = json.load(f)
with open("lora_results/results_summary.json") as f:
    lora_results = json.load(f)

In [ ]:
# Performance comparison
metrics = ['eval_accuracy', 'eval_f1_macro', 'eval_precision', 'eval_recall']
metric_names = ['Accuracy', 'F1 Score', 'Precision', 'Recall']

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for idx, (metric, name) in enumerate(zip(metrics, metric_names)):
    full_val = full_results['validation'][metric]
    full_test = full_results['test'][metric]
    lora_val = lora_results['validation'][metric]
    lora_test = lora_results['test'][metric]
    
    x = np.arange(2)
    width = 0.35
    
    axes[idx].bar(x - width/2, [full_val, full_test], width, label='Full Fine-tuning', alpha=0.8)
    axes[idx].bar(x + width/2, [lora_val, lora_test], width, label='LoRA', alpha=0.8)
    
    axes[idx].set_ylabel(name)
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(['Validation', 'Test'])
    axes[idx].legend()
    axes[idx].set_ylim([0.78, 0.88])
    axes[idx].grid(axis='y', alpha=0.3)
    
    for i, (v1, v2) in enumerate(zip([full_val, full_test], [lora_val, lora_test])):
        axes[idx].text(i - width/2, v1 + 0.003, f'{v1:.3f}', ha='center', va='bottom', fontsize=8)
        axes[idx].text(i + width/2, v2 + 0.003, f'{v2:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Parameter efficiency
trainable = lora_results['trainable_params']
total = lora_results['total_params']
frozen = total - trainable

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart
colors = ['#ff9999', '#66b3ff']
ax1.pie([trainable, frozen], labels=[f'Trainable\n{trainable:,}', f'Frozen\n{frozen:,}'], 
        colors=colors, autopct='%1.2f%%', startangle=90, explode=(0.1, 0))
ax1.set_title('LoRA Parameter Distribution', fontweight='bold')

# Bar chart
methods = ['Full Fine-tuning', 'LoRA']
params = [total, trainable]
bars = ax2.bar(methods, params, color=colors, alpha=0.8)
ax2.set_ylabel('Trainable Parameters')
ax2.set_title('Parameter Efficiency Comparison', fontweight='bold')
ax2.set_yscale('log')
ax2.grid(axis='y', alpha=0.3)

for bar, param in zip(bars, params):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
            f'{param:,}\n({param/total*100:.2f}%)', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('efficiency_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Efficiency: {(1 - trainable/total)*100:.2f}% reduction in trainable parameters")

In [ ]:
# Results table
fig, ax = plt.subplots(figsize=(10, 3.5))
ax.axis('off')

metrics = ['Accuracy', 'F1 (Macro)', 'F1 (Weighted)', 'Precision', 'Recall', 'Loss']
metric_keys = ['eval_accuracy', 'eval_f1_macro', 'eval_f1_weighted', 'eval_precision', 'eval_recall', 'eval_loss']

table_data = []
for name, key in zip(metrics, metric_keys):
    table_data.append([
        name,
        f"{full_results['validation'][key]:.4f}",
        f"{full_results['test'][key]:.4f}",
        f"{lora_results['validation'][key]:.4f}",
        f"{lora_results['test'][key]:.4f}"
    ])

table = ax.table(cellText=table_data,
                colLabels=['Metric', 'Full (Val)', 'Full (Test)', 'LoRA (Val)', 'LoRA (Test)'],
                cellLoc='center', loc='center', colWidths=[0.2, 0.15, 0.15, 0.15, 0.15])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

for i in range(5):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

for i in range(1, len(table_data) + 1):
    for j in range(5):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#f0f0f0')

plt.savefig('results_table.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Metrics heatmap
metrics = ['Accuracy', 'F1 Macro', 'Precision', 'Recall']
metric_keys = ['eval_accuracy', 'eval_f1_macro', 'eval_precision', 'eval_recall']

data = [[full_results['validation'][k], full_results['test'][k], 
         lora_results['validation'][k], lora_results['test'][k]] for k in metric_keys]

fig, ax = plt.subplots(figsize=(8, 5))
im = ax.imshow(data, cmap='YlGnBu', aspect='auto', vmin=0.80, vmax=0.88)

ax.set_xticks(np.arange(4))
ax.set_yticks(np.arange(len(metrics)))
ax.set_xticklabels(['Full (Val)', 'Full (Test)', 'LoRA (Val)', 'LoRA (Test)'])
ax.set_yticklabels(metrics)

for i in range(len(metrics)):
    for j in range(4):
        ax.text(j, i, f'{data[i][j]:.3f}', ha="center", va="center", 
                color="black", fontsize=10, fontweight='bold')

ax.set_title('Performance Metrics Heatmap', fontweight='bold', pad=15)
plt.colorbar(im, ax=ax, label='Score')
plt.tight_layout()
plt.savefig('metrics_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("\n=== Summary ===")
print(f"Full Fine-tuning Test Accuracy: {full_results['test']['eval_accuracy']:.4f}")
print(f"LoRA Test Accuracy: {lora_results['test']['eval_accuracy']:.4f}")
print(f"\nParameter Efficiency: {trainable:,} / {total:,} ({trainable/total*100:.2f}%)")
print("\n✅ Visualizations saved: metrics_comparison.png, efficiency_comparison.png, results_table.png, metrics_heatmap.png")